In [25]:
from socceraction.data.wyscout import PublicWyscoutLoader
from socceraction.spadl.wyscout import convert_to_actions as convert_to_actions_wyscout
from socceraction.spadl.statsbomb import convert_to_actions as convert_to_actions_statsbomb
from socceraction.data.opta import OptaLoader
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score, brier_score_loss, log_loss, mean_absolute_error, r2_score, mean_absolute_percentage_error, silhouette_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, balanced_accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier
import math
import pickle
import os
from name_matching.name_matcher import NameMatcher
from rapidfuzz import fuzz
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import r_regression, SelectKBest, chi2, mutual_info_classif, SequentialFeatureSelector, RFECV, SelectFromModel, mutual_info_regression, f_regression
from scipy.stats import pearsonr, chisquare
from mrmr import mrmr_classif
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import LinearSVR, SVC
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from functools import partial
from sklearn import metrics
import random
from numpy import argmax

In [26]:
# CONFIG FOR EXPERIMENTS SCENARIO
USE_EVALUATION_METRIC_CLASSIFICATION = False
SAMPLING_OPTIONS = [
    # "none",
    "random_oversampled",
    "random_undersampled",
    "smote_oversampled"
]
FEATURE_SELECTION_OPTIONS = [
    "pearson",
    "chisquare",
    "mutualinf",
    "mrmr",
    "rfembedded",
    "lasso"
]
RANDOM_STATE_OPTIONS = [0, 42]
FEATURE_SELECTION_OPTIONS_FOR_PLAYER_PROBS = [
    "pearson",
    "mutualinf"
]
if USE_EVALUATION_METRIC_CLASSIFICATION:
    MODEL_ALGORITHM_OPTIONS = [
        "xgbclassifier",
        # "catboostclassifier",
        # "rfclassifier"
    ]
else:
    MODEL_ALGORITHM_OPTIONS = [
        # "xgbregressor",
        # "rfregressor",
        "logregression"
    ]
CONFIG_EXPERIMENTS_SCENARIO_MAP = {}
CONFIG_EXPERIMENTS_SKILL_PROBS_SCENARIO_MAP = {}

def construct_config_experiments_scenario_map():
    index_counter = 1
    for random_state_opt in RANDOM_STATE_OPTIONS:
        for sampling_opt in SAMPLING_OPTIONS:
            for algorithm_opt in MODEL_ALGORITHM_OPTIONS:
                CONFIG_EXPERIMENTS_SCENARIO_MAP[index_counter] = \
                    {"sampling_opt" : sampling_opt, \
                    "algorithm_opt" : algorithm_opt, \
                    "random_state_opt" : random_state_opt}
                index_counter += 1

def construct_config_experiments_skill_probs_scenario_map():
    index_counter = 1
    for feature_selection_opt in FEATURE_SELECTION_OPTIONS_FOR_PLAYER_PROBS:
        CONFIG_EXPERIMENTS_SKILL_PROBS_SCENARIO_MAP[index_counter] = {
            "random_state_opt" : "none",
            "feature_selection_opt" : feature_selection_opt
        }
        index_counter += 1

construct_config_experiments_scenario_map()
construct_config_experiments_skill_probs_scenario_map()

In [27]:
# COLUMNS FOR TEST EXPERIMENT RESULT
COLUMNS_EVALUATION_METRIC_CLASSIFICATION = [
    "rec_score",
    "prec_score",
    "F1_score",
    "acc_score",
    "auc_score",
    "mcc_score",
    "brier_score",
    "log_loss_score",
    "balanced_acc_score"
]
COLUMNS_EVALUATION_METRIC_REGRESSION = [
    "mean_squared_error_score",
    "root_mean_squared_error_score",
    "auc_score",
    "brier_score",
    "log_loss_score",
    "mean_absolute_error_score",
    "r_squared_score",
    "mean_absolute_percentage_error_score"
]
COLUMNS_SCENARIO_NAME = [
    "sampling_opt",
    "algorithm_opt",
    "random_state_opt"
]
# if (USE_EVALUATION_METRIC_CLASSIFICATION):
#     COLUMNS_EXPERIMENT_RESULT = ["case_number"] + COLUMNS_SCENARIO_NAME + COLUMNS_EVALUATION_METRIC_CLASSIFICATION
# else:
#     COLUMNS_EXPERIMENT_RESULT = ["case_number"] + COLUMNS_SCENARIO_NAME + COLUMNS_EVALUATION_METRIC_REGRESSION

COLUMNS_EXPERIMENT_RESULT = ["case_number"] + COLUMNS_SCENARIO_NAME

COLUMNS_EXPERIMENT_RESULT_PLAYER_SKILL_PROBS = [
    "case_number",
    "feature_selection_opt",
    "random_selection_opt",
    # "mean_squared_error_score",
    # "root_mean_squared_error_score",
    # "mean_absolute_error_score",
    # "r_squared_score",
    # "mean_absolute_percentage_error_score"
]

In [28]:
api_wyscout = PublicWyscoutLoader(root="data/wyscout")
api_opta = OptaLoader(root="data/opta")
api_statsbomb = StatsBombLoader(root="data/statsbomb", getter="local")

In [29]:
for idx, action_name in enumerate(actiontypes):
    print(f'action_id : {idx}   action_name : {action_name}')

action_id : 0   action_name : pass
action_id : 1   action_name : cross
action_id : 2   action_name : throw_in
action_id : 3   action_name : freekick_crossed
action_id : 4   action_name : freekick_short
action_id : 5   action_name : corner_crossed
action_id : 6   action_name : corner_short
action_id : 7   action_name : take_on
action_id : 8   action_name : foul
action_id : 9   action_name : tackle
action_id : 10   action_name : interception
action_id : 11   action_name : shot
action_id : 12   action_name : shot_penalty
action_id : 13   action_name : shot_freekick
action_id : 14   action_name : keeper_save
action_id : 15   action_name : keeper_claim
action_id : 16   action_name : keeper_punch
action_id : 17   action_name : keeper_pick_up
action_id : 18   action_name : clearance
action_id : 19   action_name : bad_touch
action_id : 20   action_name : non_action
action_id : 21   action_name : dribble
action_id : 22   action_name : goalkick


In [30]:
for idx, bodypart_name in enumerate(bodyparts):
    print(f'bodypart_id : {idx}   bodypart_name : {bodypart_name}')

bodypart_id : 0   bodypart_name : foot
bodypart_id : 1   bodypart_name : head
bodypart_id : 2   bodypart_name : other
bodypart_id : 3   bodypart_name : head/other
bodypart_id : 4   bodypart_name : foot_left
bodypart_id : 5   bodypart_name : foot_right


In [31]:
def convert_events_df_to_spadl(events_df, home_team_id, source):
    if (source == "Statsbomb"):
        spadl_events_df = convert_to_actions_statsbomb(events_df, home_team_id)
    else:
        spadl_events_df = convert_to_actions_wyscout(events_df, home_team_id)
    spadl_events_df['time_seconds'] = spadl_events_df['time_seconds'].astype('float64')
    spadl_events_df['timestamp'] = pd.to_datetime(spadl_events_df['time_seconds'], unit='s')
    return spadl_events_df

In [32]:
# FUNCTION TO ADD ADDITIONAL INFO IN RAW SPADL DATAFRAME
STANDARD_LENGTH_COURT = 105
STANDARD_WIDTH_COURT = 68
STANDARD_GOALLINE_WIDTH = 7.32
STANDARD_LENGTH_COURT_STATSBOMB = 120
STANDARD_WIDTH_COURT_STATSBOMB = 80

# Helper Functions
def filter_out_is_home_team_apply_df(row, home_team_id):
    return 1 if row['team_id'] == home_team_id else 0

def filter_out_take_on_or_dribble_apply_df(row, take_on_action_id):
    return 1 if row['action_id'] == take_on_action_id else 0

# Helper functions specific to statsbomb opponent data
def calculate_distance_between_two_coordinates(x1, y1, x2, y2):
    return math.sqrt(abs(x2-x1) ** 2 + abs(y2-y1) ** 2)

def filter_out_non_opponent_coordinate_freeze_frame(freeze_frame_360_list):
    if (freeze_frame_360_list == None or not isinstance(freeze_frame_360_list, list)):
        return []
    return [x for x in freeze_frame_360_list if x['teammate'] == False and x['actor'] == False]

def convert_statsbomb_coordinate_to_spadl_coordinate(coordinate_x, coordinate_y):
    converted_coordinate_x = (STANDARD_LENGTH_COURT / STANDARD_LENGTH_COURT_STATSBOMB) * coordinate_x
    converted_coordinate_y = (STANDARD_WIDTH_COURT / STANDARD_WIDTH_COURT_STATSBOMB) * (STANDARD_WIDTH_COURT_STATSBOMB - coordinate_y)
    return (converted_coordinate_x, converted_coordinate_y)

def calculate_distance_opponent_apply_df(row):
    freeze_frame_360_opponents = filter_out_non_opponent_coordinate_freeze_frame(row['freeze_frame_360'])
    list_distance_opponent = []
    for object_loc in freeze_frame_360_opponents:
        opponent_x, opponent_y = convert_statsbomb_coordinate_to_spadl_coordinate(object_loc['location'][0], object_loc['location'][1])
        distance_opponent = calculate_distance_between_two_coordinates(row['start_x'], row['start_y'], opponent_x, opponent_y)
        list_distance_opponent.append(distance_opponent)
    return min(list_distance_opponent) if len(list_distance_opponent) > 0 else 0

def calculate_distance_receiver_opponent_apply_df(row):
    freeze_frame_360_opponents = filter_out_non_opponent_coordinate_freeze_frame(row['freeze_frame_360'])
    list_distance_opponent = []
    for object_loc in freeze_frame_360_opponents:
        opponent_x, opponent_y = convert_statsbomb_coordinate_to_spadl_coordinate(object_loc['location'][0], object_loc['location'][1])
        distance_opponent = calculate_distance_between_two_coordinates(row['end_x'], row['end_y'], opponent_x, opponent_y)
        list_distance_opponent.append(distance_opponent)
    return min(list_distance_opponent) if len(list_distance_opponent) > 0 else 0

def calculate_num_opponent_closer_goal(start_x, start_y, freeze_frame_360, is_home_team):
    freeze_frame_360_opponents = filter_out_non_opponent_coordinate_freeze_frame(freeze_frame_360)
    if (is_home_team):
        coordinate_x_goal = STANDARD_LENGTH_COURT
    else:
        coordinate_x_goal = 0
    coordinate_y_goal = STANDARD_WIDTH_COURT / 2

    num_opponent_closer_to_goal = 0
    for object_loc in freeze_frame_360_opponents:
        opponent_x, opponent_y = convert_statsbomb_coordinate_to_spadl_coordinate(object_loc['location'][0], object_loc['location'][1])
        distance_passer_to_goal = calculate_distance_between_two_coordinates(start_x, start_y, coordinate_x_goal, coordinate_y_goal)
        distance_opponent_to_goal = calculate_distance_between_two_coordinates(opponent_x, opponent_y, coordinate_x_goal, coordinate_y_goal)
        if (distance_opponent_to_goal < distance_passer_to_goal):
            num_opponent_closer_to_goal += 1
    return num_opponent_closer_to_goal

def calculate_num_opponent_closer_goal_apply_df(row, home_team_id):
    return calculate_num_opponent_closer_goal(row['start_x'], row['start_y'], row['freeze_frame_360'], (row['team_id'] == home_team_id))

def calculate_num_opponent_in_path(start_x, start_y, freeze_frame_360):
    path_distance = 10
    freeze_frame_360_opponents = filter_out_non_opponent_coordinate_freeze_frame(freeze_frame_360)
    num_opponent_in_path = 0
    for object_loc in freeze_frame_360_opponents:
        opponent_x, opponent_y = convert_statsbomb_coordinate_to_spadl_coordinate(object_loc['location'][0], object_loc['location'][1])
        distance_with_opponent = calculate_distance_between_two_coordinates(start_x, start_y, opponent_x, opponent_y)
        if (distance_with_opponent <= path_distance):
            num_opponent_in_path += 1
    return num_opponent_in_path

def calculate_num_opponent_in_path_apply_df(row):
    return calculate_num_opponent_in_path(row['start_x'], row['start_y'], row['freeze_frame_360'])

def calculate_num_opponent_in_path_receiver_apply_df(row):
    return calculate_num_opponent_in_path(row['end_x'], row['end_y'], row['freeze_frame_360'])

def calculate_distance_dribble(coordinate_x, coordinate_y, end_x, end_y):
    distance_passing = math.sqrt((abs(end_x - coordinate_x)) ** 2 + (abs(end_y - coordinate_y)) ** 2)
    return distance_passing

def calculate_distance_dribble_apply_df(row):
    return calculate_distance_dribble(row['start_x'], row['start_y'], row['end_x'], row['end_y'])

def calculate_num_opponent_based_on_angle_path_per_side(row, lower_limit_deg, upper_limit_deg, before_midpoint, opponent_x, opponent_y, is_actor_side):
    radius_path_length = calculate_distance_between_two_coordinates(row['start_x'], row['start_y'], row['end_x'], row['end_y'])
    if (is_actor_side):
        distance_with_opponent = calculate_distance_between_two_coordinates(row['start_x'], row['start_y'], opponent_x, opponent_y)
        distance_opponent_with_target = calculate_distance_between_two_coordinates(row['end_x'], row['end_y'], opponent_x, opponent_y)
    else:
        distance_with_opponent = calculate_distance_between_two_coordinates(row['end_x'], row['end_y'], opponent_x, opponent_y)
        distance_opponent_with_target = calculate_distance_between_two_coordinates(row['start_x'], row['start_y'], opponent_x, opponent_y)
    
    if (distance_opponent_with_target == 0) or (radius_path_length == 0):
        angle_opponent_with_path = 0
    else:
        cosine_value = (distance_with_opponent ** 2 + radius_path_length ** 2 - distance_opponent_with_target ** 2) / (2 * distance_with_opponent * radius_path_length)
        if (cosine_value > 1):
            cosine_value = 1
        elif (cosine_value < -1):
            cosine_value = -1
        angle_opponent_with_path = math.degrees(math.acos(cosine_value))

    num_opponent_in_this_angle = 0
    if (distance_with_opponent <= radius_path_length):
        if (before_midpoint and (distance_with_opponent <= (radius_path_length / 2))) or (not before_midpoint and (distance_with_opponent > (radius_path_length / 2))):
            if (angle_opponent_with_path == 0) and (upper_limit_deg == 45):
                num_opponent_in_this_angle += 1
            elif (angle_opponent_with_path > lower_limit_deg) and (angle_opponent_with_path <= upper_limit_deg):
                num_opponent_in_this_angle += 1
    return num_opponent_in_this_angle

def calculate_num_opponent_based_on_angle_path_apply_df(row, lower_limit_deg, upper_limit_deg, before_midpoint, is_actor_side):
    freeze_frame_360_opponents = filter_out_non_opponent_coordinate_freeze_frame(row['freeze_frame_360'])
    num_opponent_in_this_angle = 0
    for object_loc in freeze_frame_360_opponents:
        opponent_x, opponent_y = convert_statsbomb_coordinate_to_spadl_coordinate(object_loc['location'][0], object_loc['location'][1])
        num_opponent_in_this_angle += calculate_num_opponent_based_on_angle_path_per_side(row, lower_limit_deg, upper_limit_deg, before_midpoint, opponent_x, opponent_y, is_actor_side)
    return num_opponent_in_this_angle

# Convert horizontal start coordinate value (start_x) for away team
def convert_horizontal_start_coordinate_away_team(row, home_team_id):
    return row['start_x'] if row['team_id'] == home_team_id else (STANDARD_LENGTH_COURT - row['start_x'])

# Convert vertical start coordinate value (end_y) for away team
def convert_vertical_start_coordinate_away_team(row, home_team_id):
    return row['start_y'] if row['team_id'] == home_team_id else (STANDARD_WIDTH_COURT - row['start_y'])

# Convert horizontal end coordinate value (end_x) for away team
def convert_horizontal_end_coordinate_away_team(row, home_team_id):
    return row['end_x'] if row['team_id'] == home_team_id else (STANDARD_LENGTH_COURT - row['end_x'])

# Convert vertical start coordinate value (end_y) for away team
def convert_vertical_end_coordinate_away_team(row, home_team_id):
    return row['end_y'] if row['team_id'] == home_team_id else (STANDARD_WIDTH_COURT - row['end_y'])

# Add is_home_team column (boolean 0/1)
def add_is_home_team_column_to_spadl_df(spadl_df, home_team_id):
    spadl_df['is_home_team'] = spadl_df.apply(lambda x : filter_out_is_home_team_apply_df(x, home_team_id), axis=1)
    return spadl_df

# Add is_take_on column (boolean 0/1)
def add_is_take_on_column_to_spadl_df(spadl_df, take_on_action_id):
    spadl_df['is_take_on'] = spadl_df.apply(lambda x : filter_out_take_on_or_dribble_apply_df(x, take_on_action_id), axis=1)
    return spadl_df

# Feature 1 : distance dribble
def add_distance_dribble_to_spadl_df(spadl_df):
    spadl_df['distance_dribble'] = spadl_df.apply(calculate_distance_dribble_apply_df, axis=1)
    return spadl_df

# Feature 2 : reverse coordinate (start_x, start_y) and (end_x, end_y) for away team
def convert_start_and_end_coordinates_in_spadl_df(spadl_df, home_team_id):
    spadl_df['start_x'] = spadl_df.apply(lambda x : convert_horizontal_start_coordinate_away_team(x, home_team_id), axis=1)
    spadl_df['start_y'] = spadl_df.apply(lambda x : convert_vertical_start_coordinate_away_team(x, home_team_id), axis=1)
    spadl_df['end_x'] = spadl_df.apply(lambda x : convert_horizontal_end_coordinate_away_team(x, home_team_id), axis=1)
    spadl_df['end_y'] = spadl_df.apply(lambda x : convert_vertical_end_coordinate_away_team(x, home_team_id), axis=1)
    return spadl_df

# Opponent Feature 1 : distance opponent
def add_distance_opponent_column_to_spadl_df(spadl_df):
    spadl_df['distance_opponent'] = spadl_df.apply(calculate_distance_opponent_apply_df, axis=1)
    return spadl_df

# Opponent Feature 2 : opponents closer to goal
def add_num_opponent_closer_goal_column_to_spadl_df(spadl_df, home_team_id):
    spadl_df['num_opponent_closer_goal'] = spadl_df.apply(lambda x : calculate_num_opponent_closer_goal_apply_df(x, home_team_id), axis=1)
    return spadl_df

# Opponent Feature 3 : opponents in path
def add_num_opponent_in_path_column_to_spadl_df(spadl_df):
    spadl_df['num_opponent_in_path'] = spadl_df.apply(calculate_num_opponent_in_path_apply_df, axis=1)
    return spadl_df

# Opponent Feature 4 : distance receiver opponent
def add_distance_receiver_opponent_column_to_spadl_df(spadl_df):
    spadl_df['distance_receiver_opponent'] = spadl_df.apply(calculate_distance_receiver_opponent_apply_df, axis=1)
    return spadl_df

# Opponent Feature 5 : opponents in path receiver
def add_num_opponent_in_path_receiver_column_to_spadl_df(spadl_df):
    spadl_df['num_opponent_in_path_receiver'] = spadl_df.apply(calculate_num_opponent_in_path_receiver_apply_df, axis=1)
    return spadl_df

# Opponent Feature New 1 : add proposed features detect opponent
def add_num_opponent_based_on_angle_path_to_spadl_df(spadl_df):
    # Actor Side
    spadl_df['num_opponent_0_and_45_before_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 0, 45, True, True), axis=1)
    spadl_df['num_opponent_0_and_45_after_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 0, 45, False, True), axis=1)
    spadl_df['num_opponent_45_and_90_before_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 45, 90, True, True), axis=1)
    spadl_df['num_opponent_45_and_90_after_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 45, 90, False, True), axis=1)
    spadl_df['num_opponent_90_and_135_before_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 90, 135, True, True), axis=1)
    spadl_df['num_opponent_90_and_135_after_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 90, 135, False, True), axis=1)
    spadl_df['num_opponent_135_and_180_before_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 135, 180, True, True), axis=1)
    spadl_df['num_opponent_135_and_180_after_midpoint_actor'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 135, 180, False, True), axis=1)
    # Receiver Side
    spadl_df['num_opponent_0_and_45_before_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 0, 45, True, False), axis=1)
    spadl_df['num_opponent_0_and_45_after_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 0, 45, False, False), axis=1)
    spadl_df['num_opponent_45_and_90_before_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 45, 90, True, False), axis=1)
    spadl_df['num_opponent_45_and_90_after_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 45, 90, False, False), axis=1)
    spadl_df['num_opponent_90_and_135_before_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 90, 135, True, False), axis=1)
    spadl_df['num_opponent_90_and_135_after_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 90, 135, False, False), axis=1)
    spadl_df['num_opponent_135_and_180_before_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 135, 180, True, False), axis=1)
    spadl_df['num_opponent_135_and_180_after_midpoint_receiver'] = spadl_df.apply(lambda x : calculate_num_opponent_based_on_angle_path_apply_df(x, 135, 180, False, False), axis=1)
    
    return spadl_df

In [33]:
# Collect all dataset action specific type, export them to csv files
# Take_on (action_id = 7), Dribble (action_id = 21)
DRIBBLE_ACTION_ID = [7, 21] 
TAKE_ON_ACTION_ID = 7

def collect_raw_dribble_spadl_df(source="Wyscout", period=1):
    if source == "Statsbomb":
        api = api_statsbomb
    else:
        api = api_wyscout
    list_competitions_ids = []
    list_game_ids = []

    competitions_df = api.competitions()
    for _, row in competitions_df.iterrows():
        if source == "Statsbomb":
            if row['competition_gender'] == 'male':
                list_competitions_ids.append((row['competition_id'], row['season_id']))
        else:
            list_competitions_ids.append((row['competition_id'], row['season_id']))
        
    for competition_id, season_id in list_competitions_ids:
        games_df = api.games(competition_id, season_id)
        for _, row in games_df.iterrows():
            list_game_ids.append((row['game_id'], row['home_team_id'], row['away_team_id']))
            
    for game_id, home_team_id, away_team_id in list_game_ids:
        try:
            if (source == "Statsbomb"):
                this_game_events_df = api.events(game_id, load_360=True)
            else:
                this_game_events_df = api.events(game_id)
            this_game_events_spadl_df = convert_events_df_to_spadl(this_game_events_df, home_team_id, source)

            # Add column 360 data into events spadl data (Statsbomb)
            if (source == "Statsbomb"):
                this_game_events_spadl_df = pd.merge(this_game_events_spadl_df, this_game_events_df[["event_id", "visible_area_360", "freeze_frame_360"]], how="inner", left_on="original_event_id", right_on="event_id")
                this_game_events_spadl_df.dropna(subset=["freeze_frame_360"])
            
            # Filter action id with type dribble only, pick only data from first period
            this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['type_id'].isin(DRIBBLE_ACTION_ID)]
            # if (period != None):
            #     this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['period_id'] == period]
            # else:
            #     this_game_events_spadl_df = this_game_events_spadl_df[this_game_events_spadl_df['period_id'] == 1]
            
            # Add additional computed column to support xDribble model
            this_game_events_spadl_df = add_is_home_team_column_to_spadl_df(this_game_events_spadl_df, home_team_id)
            this_game_events_spadl_df = add_is_take_on_column_to_spadl_df(this_game_events_spadl_df, TAKE_ON_ACTION_ID)
            this_game_events_spadl_df = add_distance_dribble_to_spadl_df(this_game_events_spadl_df)
            if (source == "Statsbomb"):
                this_game_events_spadl_df = add_distance_opponent_column_to_spadl_df(this_game_events_spadl_df)
                this_game_events_spadl_df = add_num_opponent_closer_goal_column_to_spadl_df(this_game_events_spadl_df, home_team_id)
                this_game_events_spadl_df = add_num_opponent_in_path_column_to_spadl_df(this_game_events_spadl_df)
                this_game_events_spadl_df = add_num_opponent_based_on_angle_path_to_spadl_df(this_game_events_spadl_df)
                this_game_events_spadl_df = add_num_opponent_in_path_receiver_column_to_spadl_df(this_game_events_spadl_df)
                this_game_events_spadl_df = add_distance_receiver_opponent_column_to_spadl_df(this_game_events_spadl_df)
                
            # (Should be done in last step!) 
            # Convert coordinate (start_x, start_y) and (end_x, end_y) for away team
            this_game_events_spadl_df = convert_start_and_end_coordinates_in_spadl_df(this_game_events_spadl_df, home_team_id)
            
            # Export to external csv iteratively
            this_game_events_spadl_df.to_csv(f'data/training_data_xdribble/{game_id}_{home_team_id}_{away_team_id}_xdribble_data.csv')
        
        except FileNotFoundError:
            print(f'File 360 data not found {game_id}-{home_team_id}-{away_team_id}') 

In [34]:
# FUNCTIONS TO CREATE ALL DATASET PLAYERS
def collect_raw_all_players_df(source="Wyscout"):
    if (source == "Statsbomb"):
        api = api_statsbomb
    else:
        api = api_wyscout
    list_competitions_ids = []
    list_game_ids = []

    competitions_df = api.competitions()
    for _, row in competitions_df.iterrows():
        if (source == "Statsbomb"):
            if (row['competition_gender'] == 'male'):
                list_competitions_ids.append((row['competition_id'], row['season_id']))
        else:
            list_competitions_ids.append((row['competition_id'], row['season_id']))
        
    for competition_id, season_id in list_competitions_ids:
        games_df = api.games(competition_id, season_id)
        for _, row in games_df.iterrows():
            list_game_ids.append((row['game_id'], row['home_team_id'], row['away_team_id']))

    for game_id, home_team_id, away_team_id in list_game_ids:
        players_df = api.players(game_id)
        if (source == "Statsbomb"):
            players_df.to_csv(f'data/training_data_players_statsbomb/{game_id}_{home_team_id}_{away_team_id}_players_data.csv')
        else:
            players_df.to_csv(f'data/training_data_players_wyscout/{game_id}_{home_team_id}_{away_team_id}_players_data.csv')

def load_and_concat_players_df_from_csv(path_to_raw_players_df):
    list_raw_players_df = []
    for filename in os.listdir(path_to_raw_players_df):
        f = os.path.join(path_to_raw_players_df, filename)
        if os.path.isfile(f):
            players_df = pd.read_csv(f)
            list_raw_players_df.append(players_df)
    merged_players_df = pd.concat(list_raw_players_df)
    merged_players_df = merged_players_df.drop_duplicates(subset='player_id').reset_index()
    return merged_players_df

def load_csv_players_data_sofifa(path_to_sofifa_file):
    return pd.read_csv(path_to_sofifa_file)

In [35]:
# Comment it if players dataset already loaded
# collect_raw_all_players_df(source="Statsbomb")

In [36]:
# Merge wyscout player datasets with sofifa datasets by matching string name
def create_maps_for_name_matching_scores(list_unique_names_df_1, list_unique_names_df_2):
    maps_name_matching_score = {}
    for name_1 in list_unique_names_df_1:
        for name_2 in list_unique_names_df_2:
            maps_name_matching_score[(name_1, name_2)] = fuzz.ratio(name_1, name_2)
    return maps_name_matching_score

def filter_out_maps_for_name_matching_scores(maps_name_matching, threshold):
    filtered_maps_name_matching = {}
    for name_1, name_2 in maps_name_matching:
        if (maps_name_matching[(name_1, name_2)] >= threshold):
            filtered_maps_name_matching[(name_1, name_2)] = maps_name_matching[(name_1, name_2)]
    return filtered_maps_name_matching

def merge_big_dataframe_wyscout_with_sofifa(big_dataframe_players, sofifa_players_dataset, maps_name_matching_score):
    # Preprocess both dataframes and add prefix 1- and 2- to all column names to avoid duplicate column names
    big_dataframe_players.dropna(subset=['player_name'], inplace=True)
    big_dataframe_players.rename(columns=lambda x: '1-'+x, inplace=True)
    sofifa_players_dataset.dropna(subset=['full_name'], inplace=True)
    sofifa_players_dataset.rename(columns=lambda x: '2-'+x, inplace=True)
    # Merge into new empty dataframe one by one by iterating maps name matching score
    big_dataframe_players_with_sofifa = pd.DataFrame(columns=list(big_dataframe_players.columns)+list(sofifa_players_dataset.columns), index=[0])
    big_dataframe_players_with_sofifa.reset_index(inplace=True)
    for name_1, name_2 in maps_name_matching_score:
        row_from_big_dataframe_players = big_dataframe_players[big_dataframe_players['1-player_name'] == name_1].iloc[0]
        row_from_sofifa_players_dataset = sofifa_players_dataset[sofifa_players_dataset['2-full_name'] == name_2].iloc[0]
        new_row = pd.concat([row_from_big_dataframe_players, row_from_sofifa_players_dataset], axis=0, ignore_index=False)
        new_row = pd.DataFrame([new_row]).reset_index()
        big_dataframe_players_with_sofifa = pd.concat([big_dataframe_players_with_sofifa, new_row])
    # Remove prefix 1- and 2- from final big datasets
    big_dataframe_players_with_sofifa.rename(columns=lambda x: x[2:], inplace=True)
    return big_dataframe_players_with_sofifa

SOURCE_DATA = "Statsbomb"
if (SOURCE_DATA == "Statsbomb"):
    DIRECTORY_PLAYERS_CSV_DATAS = "data/training_data_players_statsbomb"
else:
    DIRECTORY_PLAYERS_CSV_DATAS = "data/training_data_players_wyscout"
DIRECTORY_SOFIFA_CSV_DATAS = "data/players_skill_dataset/sofifa_dataset_cleaned.csv"
DIRECTORY_WYSCOUT_CSV_DATAS = "data/players_skill_dataset/wyscout_dataset_cleaned.csv"
DIRECTORY_STATSBOMB_CSV_DATAS = "data/players_skill_dataset/statsbomb_dataset_cleaned.csv"
DIRECTORY_FINAL_PLAYERS_CSV_DATAS = "data/players_skill_dataset/final_players_skill_dataset.csv"

# COMMENT BELOW SNIPPET CODES IF FINAL PLAYER DATASETS WITH SKILL ALREADY GENERATED !!
# big_dataframe_players = load_and_concat_players_df_from_csv(DIRECTORY_PLAYERS_CSV_DATAS)
# if (SOURCE_DATA == "Statsbomb"):
#     big_dataframe_players.to_csv(DIRECTORY_STATSBOMB_CSV_DATAS)
# else:
#     big_dataframe_players.to_csv(DIRECTORY_WYSCOUT_CSV_DATAS)
# sofifa_players_dataset = load_csv_players_data_sofifa(DIRECTORY_SOFIFA_CSV_DATAS)

# maps_name_matching_score = create_maps_for_name_matching_scores(big_dataframe_players['player_name'].unique(), sofifa_players_dataset['full_name'].unique())
# maps_name_matching_score = filter_out_maps_for_name_matching_scores(maps_name_matching_score, threshold=70)

# big_dataframe_players_with_sofifa = merge_big_dataframe_wyscout_with_sofifa(big_dataframe_players, sofifa_players_dataset, maps_name_matching_score)
# big_dataframe_players_with_sofifa.reset_index(inplace=True)
# big_dataframe_players_with_sofifa = big_dataframe_players_with_sofifa.drop_duplicates(subset='player_id')
# big_dataframe_players_with_sofifa.to_csv(DIRECTORY_FINAL_PLAYERS_CSV_DATAS)

In [37]:
# MAIN DRIVER (comment it if csv files already loaded)
# collect_raw_dribble_spadl_df(source="Statsbomb")

In [38]:
# Load csv datas already retrieved then concat them into one big dataframe
DIRECTORY_XDRIBBLE_CSV_DATAS = "data/training_data_xdribble"

def load_and_concat_xdribble_df_from_csv():
    list_pass_event_df = []
    for filename in os.listdir(DIRECTORY_XDRIBBLE_CSV_DATAS):
        f = os.path.join(DIRECTORY_XDRIBBLE_CSV_DATAS, filename)
        if os.path.isfile(f):
            pass_event_df = pd.read_csv(f)
            list_pass_event_df.append(pass_event_df)
    return pd.concat(list_pass_event_df)

In [39]:
# JOIN ALREADY CONSTRUCTED PLAYER SKILLS DATASET WITH ORIGIN EVENT DATASET WYSCOUT
player_skills_dataset = pd.read_csv(DIRECTORY_FINAL_PLAYERS_CSV_DATAS)
big_dataframe_xdribble_model = load_and_concat_xdribble_df_from_csv()
# big_dataframe_xdribble_model = big_dataframe_xdribble_model.merge(player_skills_dataset, how='inner',on='player_id')
big_dataframe_xdribble_model.head()

,Unnamed: 0,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,...,num_opponent_0_and_45_before_midpoint_receiver,num_opponent_0_and_45_after_midpoint_receiver,num_opponent_45_and_90_before_midpoint_receiver,num_opponent_45_and_90_after_midpoint_receiver,num_opponent_90_and_135_before_midpoint_receiver,num_opponent_90_and_135_after_midpoint_receiver,num_opponent_135_and_180_before_midpoint_receiver,num_opponent_135_and_180_after_midpoint_receiver,num_opponent_in_path_receiver,distance_receiver_opponent
0,1,3788741,bea4235d-7e40-461c-bb82-6d473f5bb324,1,2.0,909,8963.0,27.794118,44.070886,31.058824,...,0,0,0,0,0,0,0,0,0,11.749472
1,3,3788741,1615d483-101d-45c1-b554-04cf69897232,1,6.0,909,8541.0,61.500000,3.615190,61.235294,...,0,0,0,0,0,0,0,0,1,1.155704
2,6,3788741,4cbd1088-1580-43f5-8b09-58efabb753ff,1,14.0,914,6954.0,13.058824,62.405063,2.029412,...,0,0,0,0,0,0,0,0,0,69.368778
3,8,3788741,58ef2997-552a-4d3e-854a-cf863f9f5b71,1,18.0,914,7036.0,6.088235,37.701266,7.147059,...,0,0,0,0,0,0,0,0,0,63.611970
4,10,3788741,5eae5d7a-59cb-42e0-bfc0-c5a2e80493ee,1,21.0,914,7173.0,16.500000,27.630380,24.529412,...,0,0,0,0,0,0,0,0,0,38.229349


In [40]:
# SELECT ONLY FEATURED COLUMN FROM BIG DATASETS
# features_column_included = ["player_id", "original_event_id", "start_x", "start_y", "end_x", "end_y", "distance_opponent", 
#                             "num_opponent_closer_goal", "num_opponent_in_path", "result_id"]
features_column_included = ["player_id", "original_event_id", "distance_opponent", 
                            "distance_receiver_opponent", "result_id",
                            "num_opponent_in_path", "num_opponent_in_path_receiver",
                            "num_opponent_0_and_45_before_midpoint_actor",
                            "num_opponent_0_and_45_after_midpoint_actor",
                            "num_opponent_0_and_45_before_midpoint_receiver",
                            "num_opponent_0_and_45_after_midpoint_receiver"]
player_skills_column_included = ["acceleration", "aggression", "agility", "balance", "ball_control",
                                 "composure", "crossing", "curve", "dribbling", "finishing",
                                 "freekick_accuracy", "heading_accuracy", "interceptions", "jumping", "long_passing",
                                 "long_shots", "marking", "penalties", "positioning", "reactions",
                                 "shot_power", "sliding_tackle", "sprint_speed", "stamina", "short_passing",
                                 "standing_tackle", "strength", "vision", "volleys"]
player_attribute_column_included = ["height_cm", "weight_kgs", "age"]

big_dataframe_xdribble_model = big_dataframe_xdribble_model[[c for c in big_dataframe_xdribble_model.columns if c in (features_column_included + player_skills_column_included + player_attribute_column_included)]]
big_dataframe_xdribble_model.head()

,original_event_id,player_id,result_id,distance_opponent,num_opponent_in_path,num_opponent_0_and_45_before_midpoint_actor,num_opponent_0_and_45_after_midpoint_actor,num_opponent_0_and_45_before_midpoint_receiver,num_opponent_0_and_45_after_midpoint_receiver,num_opponent_in_path_receiver,distance_receiver_opponent
0,bea4235d-7e40-461c-bb82-6d473f5bb324,8963.0,1,15.201729,0,0,0,0,0,0,11.749472
1,1615d483-101d-45c1-b554-04cf69897232,8541.0,1,0.896092,1,0,0,0,0,1,1.155704
2,4cbd1088-1580-43f5-8b09-58efabb753ff,6954.0,1,61.133246,0,0,0,0,0,0,69.368778
3,58ef2997-552a-4d3e-854a-cf863f9f5b71,7036.0,1,64.844639,0,0,0,0,0,0,63.611970
4,5eae5d7a-59cb-42e0-bfc0-c5a2e80493ee,7173.0,1,44.956121,0,0,0,0,0,0,38.229349


In [41]:
# LOAD PROBABILITY SKILL DATASETS FOR DRIBBLING EVENT
DIRECTORY_PLAYER_SKILLS_PROBABILITIES_DATAS = "data/model_xdribble/xdribble_player_skill_probs_dataset.csv"

xdribble_player_skill_probs_df = pd.read_csv(DIRECTORY_PLAYER_SKILLS_PROBABILITIES_DATAS)
xdribble_player_skill_probs_df_copy = xdribble_player_skill_probs_df.copy()

# Scaling all feature columns in range (0, 1)
xdribble_player_skill_probs_df_copy[player_skills_column_included] = xdribble_player_skill_probs_df_copy[player_skills_column_included] / 100

# Filtering outliers data based on DBSCAN Plot Image (see generator code)
epsilon = 0.5
min_samples = 62
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
dbscan.fit(xdribble_player_skill_probs_df_copy[player_skills_column_included + ["statistic_success_action_among_players", "statistic_success_action_probs"]])
labels = dbscan.labels_
outliers_data = xdribble_player_skill_probs_df_copy[labels == -1]

xdribble_player_skill_probs_df = xdribble_player_skill_probs_df[~xdribble_player_skill_probs_df.index.isin(outliers_data.index)]
xdribble_player_skill_probs_df[["full_name", "statistic_success_action_probs", "statistic_success_action_among_players", "statistic_success_total"]].sort_values(by="statistic_success_action_probs", ascending=False)

,full_name,statistic_success_action_probs,statistic_success_action_among_players,statistic_success_total
1058,Alejandro Remiro Gargallo,1.000000,0.000126,0.000126
996,Sammy Ameobi,1.000000,0.000004,0.000004
366,Dominique Heintz,1.000000,0.000003,0.000003
365,Jamie Murphy,1.000000,0.000027,0.000027
364,Romain Cagnon,1.000000,0.000017,0.000017
...,...,...,...,...
1112,Manuel Jesús Vázquez Florido,0.846154,0.000014,0.000012
108,Tobias Sana,0.842105,0.000021,0.000017
1003,Mathias Jensen,0.833333,0.000019,0.000016
1266,Alfa Semedo Esteves,0.800000,0.000005,0.000004


In [42]:
# CASE 1 : Random Oversample Function
def training_data_random_oversampled(X_train, Y_train, random_state):
    ros = RandomOverSampler(random_state=random_state)
    X_resampled, Y_resampled = ros.fit_resample(X_train, Y_train)
    return (X_resampled, Y_resampled)

# CASE 2 : Random Undersample Function
def training_data_random_undersampled(X_train, Y_train, random_state):
    rus = RandomUnderSampler(random_state=random_state)
    X_resampled, Y_resampled = rus.fit_resample(X_train, Y_train)
    return (X_resampled, Y_resampled)

# CASE 3 : Random SMOTE Oversample Function
def training_data_smote_oversampled(X_train, Y_train, random_state):
    X_resampled, Y_resampled = SMOTE(random_state=random_state).fit_resample(X_train, Y_train)
    return (X_resampled, Y_resampled)

# V CASE 1 : Feature Selection - Pearson Coefficient
def filter_columns_feature_selection_pearson(X_train, Y_train, columns_considered, threshold):
    new_columns_after_selection = []
    for _, skill in enumerate(columns_considered):
        correlation_value, _ = pearsonr(X_train[skill], Y_train)
        if correlation_value >= threshold:
            new_columns_after_selection.append(skill)
    return new_columns_after_selection

def training_data_feature_selection_pearson(X_train, Y_train, columns_considered, threshold):
    columns_selected = filter_columns_feature_selection_pearson(X_train, Y_train, columns_considered, threshold)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# V CASE 2 : Feature Selection - Chi Square
def filter_columns_feature_selection_chisquare(X_train, Y_train, columns_considered, num_of_features):
    chi2_selector = SelectKBest(chi2, k=num_of_features) 
    df_feature = X_train[columns_considered]
    chi2_selector.fit(df_feature, Y_train)
    cols = chi2_selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_chisquare(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_chisquare(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# V CASE 3 : Feature Selection - Mutual Information
def filter_columns_feature_selection_mutualinf(X_train, Y_train, columns_considered, num_of_features):
    mi_selector = SelectKBest(mutual_info_classif, k=num_of_features) 
    df_feature = X_train[columns_considered]
    mi_selector.fit(df_feature, Y_train)
    cols = mi_selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_mutualinf(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_mutualinf(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# V CASE 4 : Feature Selection - mRMR Selection
def filter_columns_feature_selection_mrmr(X_train, Y_train, columns_considered, num_of_features):
    df_feature = X_train[columns_considered]
    selected_features = mrmr_classif(X=df_feature, y=Y_train, K=num_of_features)
    return selected_features

def training_data_feature_selection_mrmr(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_mrmr(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# X CASE 5 : Feature Selection - Sequential Forward Selection (SFS)
def filter_columns_feature_selection_sfs(X_train, Y_train, columns_considered, num_of_features):
    rf = RandomForestClassifier()
    sfs = SequentialFeatureSelector(rf, n_features_to_select=num_of_features, direction='forward')
    df_feature = X_train[columns_considered]
    sfs.fit(df_feature, Y_train)
    cols = sfs.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_sfs(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_sfs(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# X CASE 6 : Feature Selection - Sequential Backward Elimination (SBE)
def filter_columns_feature_selection_sbe(X_train, Y_train, columns_considered, num_of_features):
    rf = RandomForestClassifier()
    sfs = SequentialFeatureSelector(rf, n_features_to_select=num_of_features, direction='backward')
    df_feature = X_train[columns_considered]
    sfs.fit(df_feature, Y_train)
    cols = sfs.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_sbe(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_sbe(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# X CASE 7 : Feature Selection - Recursive Feature Elimination
def filter_columns_feature_selection_rfe(X_train, Y_train, columns_considered, num_of_features):
    estimator = LinearSVR()
    selector = RFECV(estimator, step=1, cv=num_of_features)
    df_feature = X_train[columns_considered]
    selector.fit(df_feature, Y_train)
    cols = selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_rfe(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_rfe(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# V CASE 8 : Feature Selection - Random Forest Embedded (rfembedded)
def filter_columns_feature_selection_rfembedded(X_train, Y_train, columns_considered, num_of_features):
    estimator = RandomForestClassifier()
    selector = SelectFromModel(estimator=estimator, max_features=num_of_features)
    df_feature = X_train[columns_considered]
    selector.fit(df_feature, Y_train)
    cols = selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_rfembedded(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_rfembedded(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# V CASE 9 : Feature Selection - LASSO
def filter_columns_feature_selection_lasso(X_train, Y_train, columns_considered, num_of_features):
    estimator = LogisticRegression(penalty='l2', C=0.5, solver='newton-cholesky')
    selector = SelectFromModel(estimator=estimator, max_features=num_of_features)
    df_feature = X_train[columns_considered]
    selector.fit(df_feature, Y_train)
    cols = selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_lasso(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_lasso(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# CASE 1 : Train with model XGBRegressor
def fit_and_train_with_model_xgbregressor(X_train, Y_train):
    model = XGBRegressor(objective="reg:logistic")
    model.fit(X_train, Y_train)
    return model

# CASE 2 : Train with model RandomForestRegressor
def fit_and_train_with_model_rfregressor(X_train, Y_train):
    model = RandomForestRegressor()
    model.fit(X_train, Y_train)
    return model

# CASE 3 : Train with model LogisticRegression
def fit_and_train_with_model_logregression(X_train, Y_train, random_state_opt):
    model = LogisticRegression(random_state=random_state_opt)
    model.fit(X_train, Y_train)
    return model

# CASE 4 : Train with model XGBClassifier
def fit_and_train_with_model_xgbclassifier(X_train, Y_train):
    # model = XGBClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbosity=1, enable_categorical=True)
    model = XGBClassifier(objective="binary:logistic")
    model.fit(X_train, Y_train)
    return model

# CASE 5 : Train with model Catboost Classifier 
def fit_and_train_with_model_catboostclassifier(X_train, Y_train):
    # model = CatBoostClassifier(n_estimators=50, max_depth=3, verbose=1)
    model = CatBoostClassifier()
    model.fit(X_train, Y_train)
    return model

# CASE 6 : Train with model RandomForest Classifier
def fit_and_train_with_model_rfclassifier(X_train, Y_train):
    # model = RandomForestClassifier(n_estimators=50, max_depth=3, n_jobs=-3, verbose=1)
    model = RandomForestClassifier()
    model.fit(X_train, Y_train)
    return model

In [43]:
# # FEATURE PREPROCESSING BIG DATASETS AND CREATE XGBOOST MODEL
# # 1. Change all numeric columns with MinMaxScaler
# scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
# # columns_minmax_scaler = ["start_x", "start_y", "end_x", "end_y", "distance_opponent", 
# #                         "num_opponent_closer_goal", "num_opponent_in_path"]
# columns_minmax_scaler = ["distance_opponent", "num_opponent_in_path",
#                         "distance_receiver_opponent", "num_opponent_in_path_receiver",
#                         "num_opponent_0_and_45_before_midpoint_actor",
#                         "num_opponent_0_and_45_after_midpoint_actor",
#                         "num_opponent_0_and_45_before_midpoint_receiver",
#                         "num_opponent_0_and_45_after_midpoint_receiver"]
# # Store Description for all Numeric Columns in External CSV
# df_description_numeric = big_dataframe_xdribble_model[columns_minmax_scaler].describe()
# filename = 'xdribble_description_numeric_data.csv'
# directory_model = "data/model_xdribble/"
# df_description_numeric.to_csv(directory_model + filename)
# # Preprocess to minmax scaler
# big_dataframe_xdribble_model[columns_minmax_scaler] = scaler.fit_transform(big_dataframe_xdribble_model[columns_minmax_scaler])

# # 2. Check if data is unbalanced. If it is unbalanced, then do method to oversize the sample
# print(big_dataframe_xdribble_model['result_id'].value_counts())

# # 3. Change result_id label into float64 type
# # big_dataframe_xdribble_model['result_id'] = big_dataframe_xdribble_model['result_id'].astype('float64')

# # 4. Remove dataframe instead of having result_id (0,1) --> (fail, success)
# big_dataframe_xdribble_model = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'].isin([0,1])]
# print(big_dataframe_xdribble_model['result_id'].value_counts())

# # 5. Split train data and test data from Big Datasets
# big_dataframe_xdribble_model.reset_index(inplace=True, drop=True)
# # sample_label_0 = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'] == 0]
# # sample_label_1 = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'] == 1]
# # big_dataframe_xdribble_model_test = pd.concat([sample_label_0.sample(frac=0.2, random_state=42), sample_label_1.sample(frac=0.2, random_state=42)])
# # big_dataframe_xdribble_model = big_dataframe_xdribble_model[~big_dataframe_xdribble_model.index.isin(big_dataframe_xdribble_model_test.index)]
# # print(big_dataframe_xdribble_model_test['result_id'].value_counts())
# print(big_dataframe_xdribble_model['result_id'].value_counts())

# # filename = "xdribble_dataset_for_accuracy_test.csv"
# # directory_model = "data/model_xdribble/"
# # big_dataframe_xdribble_model.to_csv(directory_model + filename)

# all_feature_columns = columns_minmax_scaler
# X_train = big_dataframe_xdribble_model[all_feature_columns]
# Y_train = big_dataframe_xdribble_model["result_id"]

# # Empty dataframe for saving test result
# empty_test_result = pd.DataFrame(columns=COLUMNS_EXPERIMENT_RESULT, index=[0])

# for case_number in sorted(list(CONFIG_EXPERIMENTS_SCENARIO_MAP.keys())):
#     sampling_opt = CONFIG_EXPERIMENTS_SCENARIO_MAP[case_number]["sampling_opt"]
#     # feature_selection_opt = CONFIG_EXPERIMENTS_SCENARIO_MAP[case_number]["feature_selection_opt"]
#     algorithm_opt = CONFIG_EXPERIMENTS_SCENARIO_MAP[case_number]["algorithm_opt"]
#     random_state_opt = CONFIG_EXPERIMENTS_SCENARIO_MAP[case_number]["random_state_opt"]

#     # 6. Do oversampling/undersampling and feature selection at same time
#     if sampling_opt == "none":
#         X_resampled, Y_resampled = X_train, Y_train
#     else:
#         X_resampled, Y_resampled = globals()["training_data_" + sampling_opt](X_train, Y_train, random_state_opt)
#     # if feature_selection_opt == "pearson":
#     #     threshold = 0.5
#     #     X_feature_sel, Y_feature_sel = globals()["training_data_feature_selection_" + feature_selection_opt](X_resampled, Y_resampled, player_skills_column_included, threshold)
#     # else:
#     #     num_of_features = 10
#     #     X_feature_sel, Y_feature_sel = globals()["training_data_feature_selection_" + feature_selection_opt](X_resampled, Y_resampled, player_skills_column_included, num_of_features)

#     # # 7. Do train_test_split on training data
#     # X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_resampled, Y_resampled, test_size=0.2, random_state=random_state_opt)

#     # 8. Train Model
#     model = globals()["fit_and_train_with_model_" + algorithm_opt](X_resampled, Y_resampled, random_state_opt)

#     # # 9. Predict Testing Data
#     # y_predict = model.predict(X_test_split)

#     # # 10. Save test result experiment
#     # if (USE_EVALUATION_METRIC_CLASSIFICATION):
#     #     rec_score = recall_score(y_test_split, y_predict)
#     #     prec_score = precision_score(y_test_split, y_predict)
#     #     F1_score = f1_score(y_test_split, y_predict)
#     #     acc_score = accuracy_score(y_test_split, y_predict)
#     #     auc_score = roc_auc_score(y_test_split, y_predict)
#     #     mcc_score = matthews_corrcoef(y_test_split, y_predict)
#     #     brier_score = brier_score_loss(y_test_split, y_predict)
#     #     log_loss_score = log_loss(y_test_split, y_predict)
#     #     balanced_acc_score = balanced_accuracy_score(y_test_split, y_predict)
#     # else:
#     #     mean_squared_error_score = mean_squared_error(y_test_split, y_predict)
#     #     root_mean_squared_error_score = mean_squared_error(y_test_split, y_predict, squared=False)
#     #     auc_score = roc_auc_score(y_test_split, y_predict)
#     #     brier_score = brier_score_loss(y_test_split, y_predict)
#     #     log_loss_score = log_loss(y_test_split, y_predict)
#     #     mean_absolute_error_score = mean_absolute_error(y_test_split, y_predict)
#     #     r_squared_score = r2_score(y_test_split, y_predict)
#     #     mean_absolute_percentage_error_score = mean_absolute_percentage_error(y_test_split, y_predict)

#     maps_new_row = {}
#     if USE_EVALUATION_METRIC_CLASSIFICATION:
#         eval_metrics_column = COLUMNS_EVALUATION_METRIC_CLASSIFICATION
#     else:
#         eval_metrics_column = COLUMNS_EVALUATION_METRIC_REGRESSION
#     for column in COLUMNS_EXPERIMENT_RESULT:
#         if column not in eval_metrics_column:
#             if column == "case_number":
#                 maps_new_row["case_number"] = case_number
#             elif column in COLUMNS_SCENARIO_NAME:
#                 maps_new_row[column] = globals()[column]
#         else:
#             maps_new_row[column] = globals()[column]     
#     new_row = pd.DataFrame(maps_new_row, index=[0])
#     empty_test_result = pd.concat([new_row, empty_test_result.loc[:]]).reset_index(drop=True)

#     # 11. Save model to external file
#     filename = f'xdribble_model_case_{case_number}.sav'
#     directory_model = "data/model_xdribble/"
#     pickle.dump(model, open(directory_model + filename, 'wb'))

# # 12. Save test result experiment to external file
# filename = 'xdribble_test_model_experiment_result.csv'
# directory_model = "data/model_xdribble/"
# empty_test_result.to_csv(directory_model + filename)

In [44]:
# CASE 1 : Feature Selection for Regression - Mutual Information
def filter_columns_feature_selection_reg_mutualinf(X_train, Y_train, columns_considered, num_of_features):
    mi_selector = SelectKBest(score_func=partial(mutual_info_regression, random_state=0), k=num_of_features)
    df_feature = X_train[columns_considered]
    mi_selector.fit(df_feature, Y_train)
    cols = mi_selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_reg_mutualinf(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_reg_mutualinf(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# CASE 2 : Feature Selection for Regression - Pearson Coefficient
def filter_columns_feature_selection_reg_pearson(X_train, Y_train, columns_considered, num_of_features):
    mi_selector = SelectKBest(f_regression, k=num_of_features) 
    df_feature = X_train[columns_considered]
    mi_selector.fit(df_feature, Y_train)
    cols = mi_selector.get_support(indices=True)
    df_selected_features = df_feature.iloc[:,cols]
    return df_selected_features.columns

def training_data_feature_selection_reg_pearson(X_train, Y_train, columns_considered, num_of_features):
    columns_selected = filter_columns_feature_selection_reg_pearson(X_train, Y_train, columns_considered, num_of_features)
    columns_omitted = [x for x in columns_considered if x not in columns_selected]
    final_columns = [x for x in list(X_train.columns) if x not in columns_omitted]
    print(final_columns)
    return (X_train[final_columns], Y_train)

# CASE 1 : Train with model LinearRegressor
def fit_and_train_with_model_linearregressor(X_train, Y_train):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    return model

In [45]:
# # CREATE REGRESSION XGBOOST MODEL FOR PLAYER SKILL PROBABILITIES DATASET 
# # 1. Change all numeric columns with MinMaxScaler
# scaler = preprocessing.StandardScaler()
# columns_minmax_scaler = player_skills_column_included + player_attribute_column_included + ["statistic_success_action_probs"]
# # Store Description for all Numeric Columns in External CSV
# df_description_numeric = xdribble_player_skill_probs_df[columns_minmax_scaler].describe()
# filename = 'xdribble_skill_probs_description_numeric_data.csv'
# directory_model = "data/model_xdribble/"
# df_description_numeric.to_csv(directory_model + filename)
# # Preprocess to minmax scaler
# xdribble_player_skill_probs_df[columns_minmax_scaler] = scaler.fit_transform(xdribble_player_skill_probs_df[columns_minmax_scaler])

# # 2. Change type of minmax column as float64
# xdribble_player_skill_probs_df[columns_minmax_scaler] = xdribble_player_skill_probs_df[columns_minmax_scaler].astype('float64')

# # 3. Split train data and test data from Big Datasets
# all_feature_columns = player_skills_column_included + player_attribute_column_included
# X_train = xdribble_player_skill_probs_df[all_feature_columns]
# Y_train = xdribble_player_skill_probs_df["statistic_success_action_probs"]

# # Empty dataframe for saving test result
# empty_test_result = pd.DataFrame(columns=COLUMNS_EXPERIMENT_RESULT_PLAYER_SKILL_PROBS, index=[0])

# for case_number in sorted(list(CONFIG_EXPERIMENTS_SKILL_PROBS_SCENARIO_MAP.keys())):
#     feature_selection_opt = CONFIG_EXPERIMENTS_SKILL_PROBS_SCENARIO_MAP[case_number]["feature_selection_opt"]
#     random_state_opt = CONFIG_EXPERIMENTS_SKILL_PROBS_SCENARIO_MAP[case_number]["random_state_opt"]

#     # 4. Do feature selection on train data
#     num_of_features = 10
#     X_feature_sel, Y_feature_sel = globals()["training_data_feature_selection_reg_" + feature_selection_opt](X_train, Y_train, player_skills_column_included, num_of_features)

#     # # 5. Do train_test_split on training data
#     # X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_feature_sel, Y_feature_sel, test_size=0.2, random_state=random_state_opt)

#     # 6. Train Model with XGBRegressor
#     model = fit_and_train_with_model_linearregressor(X_feature_sel, Y_feature_sel)

#     # # 7. Predict Testing Data
#     # y_predict = model.predict(X_test_split)

#     # # 8. Save test result experiment
#     # mean_squared_error_score = mean_squared_error(y_test_split, y_predict)
#     # root_mean_squared_error_score = mean_squared_error(y_test_split, y_predict, squared=False)
#     # mean_absolute_error_score = mean_absolute_error(y_test_split, y_predict)
#     # r_squared_score = r2_score(y_test_split, y_predict)
#     # mean_absolute_percentage_error_score = mean_absolute_percentage_error(y_test_split, y_predict)

#     maps_new_row = {}
#     eval_metrics_column = COLUMNS_EVALUATION_METRIC_REGRESSION
#     for column in COLUMNS_EXPERIMENT_RESULT_PLAYER_SKILL_PROBS:
#         if column not in eval_metrics_column:
#             if column == "feature_selection_opt":
#                 maps_new_row["feature_selection_opt"] = feature_selection_opt
#             elif column == "random_state_opt":
#                 maps_new_row["random_state_opt"] = random_state_opt
#             elif column == "case_number":
#                 maps_new_row["case_number"] = case_number
#         else:
#             maps_new_row[column] = globals()[column]     
#     new_row = pd.DataFrame(maps_new_row, index=[0])
#     empty_test_result = pd.concat([new_row, empty_test_result.loc[:]]).reset_index(drop=True)

#     # 9. Save model to external file
#     filename = f'xdribble_player_skill_probs_model_case_{case_number}.sav'
#     directory_model = "data/model_xdribble/"
#     pickle.dump(model, open(directory_model + filename, 'wb'))

# # 10. Save test result experiment to external file
# filename = 'xdribble_player_skill_probs_model_experiment_result.csv'
# directory_model = "data/model_xdribble/"
# empty_test_result.to_csv(directory_model + filename)

In [46]:
# CALCULATE REAL ACCURACY OF SUCCESSFUL DRIBBLING BASED ON THRESHOLD
CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL = [2]
CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL = [2]
LIST_DEFINED_THRESHOLD = [1.201546250730531e-09]
MAPS_ACCURACY_SCORE_XDRIBBLE_MODEL = {}
MAPS_PRECISION_SCORE_XDRIBBLE_MODEL = {}
MAPS_RECALL_SCORE_XDRIBBLE_MODEL = {}
MAPS_F_SCORE_XDRIBBLE_MODEL = {}
MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID = {}
MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID = {}
COLUMNS_XDRIBBLE_FINAL_ACCURACY_RESULT = [
    "case_number_xdribble_model",
    "case_number_xdribble_skill_probs_model",
    "accuracy_without_skill",
    "accuracy_with_player_skill",
    "accuracy_difference",
    "precision_without_skill",
    "precision_with_player_skill",
    "precision_difference",
    "recall_without_skill",
    "recall_with_player_skill",
    "recall_difference",
    "f_score_without_skill",
    "f_score_with_player_skill",
    "f_score_difference"
]
COLUMNS_XDRIBBLE_CONFUSION_MATRIX_DISTRIBUTION_RESULT = [
    "case_number_xdribble_model",
    "case_number_xdribble_skill_probs_model",
    "true_positive",
    "true_negative",
    "false_positive",
    "false_negative"
]
DIRECTORY_MODEL = "data/model_xdribble/"
FILENAME_DATA_TEST = "xdribble_dataset_for_accuracy_test.csv"

# Difficulty Value : Change all numeric columns with StandardScaler
scaler = preprocessing.StandardScaler()
columns_minmax_scaler = ["distance_opponent", "num_opponent_in_path",
                        "distance_receiver_opponent", "num_opponent_in_path_receiver",
                        "num_opponent_0_and_45_before_midpoint_actor",
                        "num_opponent_0_and_45_after_midpoint_actor",
                        "num_opponent_0_and_45_before_midpoint_receiver",
                        "num_opponent_0_and_45_after_midpoint_receiver"]
big_dataframe_xdribble_model[columns_minmax_scaler] = scaler.fit_transform(big_dataframe_xdribble_model[columns_minmax_scaler])
big_dataframe_xdribble_model = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'].isin([0,1])]

# Player Skill Probabilities : Change all numeric columns with MinMaxScaler
scaler = preprocessing.StandardScaler()
columns_minmax_scaler = player_skills_column_included + player_attribute_column_included + ["statistic_success_action_probs"]
xdribble_player_skill_probs_df[columns_minmax_scaler] = scaler.fit_transform(xdribble_player_skill_probs_df[columns_minmax_scaler])

xdribble_data_test = big_dataframe_xdribble_model

for threshold in LIST_DEFINED_THRESHOLD:
    DEFINED_THRESHOLD = threshold
    # Construct maps xdribble difficulty values and maps accuracy score xdribble model
    for case_number_xdribble_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL:
        # Loads xdribble difficulty model
        filename_xdribble_model = f'xdribble_model_case_{case_number_xdribble_model}.sav'
        xdribble_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_model, 'rb'))
        feature_names_xdribble_model = xdribble_model.feature_names_in_

        actual_result_ids = pd.Series(xdribble_data_test["result_id"]).tolist()
        original_event_ids = pd.Series(xdribble_data_test["original_event_id"]).tolist()
        player_ids = pd.Series(xdribble_data_test["player_id"]).tolist()
        xdribble_train_data = xdribble_data_test[feature_names_xdribble_model]
        xdribble_difficulty_value = [p[1] for p in xdribble_model.predict_proba(xdribble_train_data)]
        
        TP_counter, FP_counter, FN_counter, TN_counter = 0, 0, 0, 0
        for idx, event_id in enumerate(original_event_ids):
            actual_result_id = actual_result_ids[idx]
            xdribble_difficulty_this_value = xdribble_difficulty_value[idx]
            player_id = player_ids[idx]
            player_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == player_id]
            if (not player_data.empty):
                MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID[(case_number_xdribble_model, event_id)] = xdribble_difficulty_this_value
                predicted_result_id = 1 if xdribble_difficulty_this_value >= DEFINED_THRESHOLD else 0
                if (actual_result_id == 1) and (predicted_result_id == 1):
                    TP_counter += 1
                elif (actual_result_id == 0) and (predicted_result_id == 1):
                    FP_counter += 1
                elif (actual_result_id == 1) and (predicted_result_id == 0):
                    FN_counter += 1
                elif (actual_result_id == 0) and (predicted_result_id == 0):
                    TN_counter += 1
        MAPS_ACCURACY_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model] = (TP_counter + TN_counter) / (TP_counter + TN_counter + FP_counter + FN_counter)
        MAPS_PRECISION_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model] = TP_counter / (TP_counter + FP_counter)
        MAPS_RECALL_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model] = TP_counter / (TP_counter + FN_counter)
        MAPS_F_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model] = TP_counter / (TP_counter + 0.5 * (FP_counter + FN_counter))

    # Construct maps xdribble player skill probabilities value
    unique_player_ids = list(set(pd.Series(xdribble_data_test["player_id"]).tolist()))
    for case_number_xdribble_skill_probs_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL:
        # Load xdribble player skill probs model
        filename_xdribble_skill_probs_model = f'xdribble_player_skill_probs_model_case_{case_number_xdribble_skill_probs_model}.sav'
        xdribble_skill_probs_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_skill_probs_model, 'rb'))
        feature_names_xdribble_skill_probs_model = xdribble_skill_probs_model.feature_names_in_

        for player_id in unique_player_ids:
            xdribble_skill_train_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == player_id][feature_names_xdribble_skill_probs_model]
            if (not xdribble_skill_train_data.empty):
                xdribble_skill_value = xdribble_skill_probs_model.predict(xdribble_skill_train_data)[0]
                xdribble_skill_value = 1 / (1 + math.exp(-1 * xdribble_skill_value))
                MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID[(case_number_xdribble_skill_probs_model, player_id)] = xdribble_skill_value

    # Final Calculation and Saving Test Result
    empty_test_result = pd.DataFrame(columns=COLUMNS_XDRIBBLE_FINAL_ACCURACY_RESULT, index=[0])
    empty_confusion_matrix_result = pd.DataFrame(columns=COLUMNS_XDRIBBLE_CONFUSION_MATRIX_DISTRIBUTION_RESULT, index=[0])

    for case_number_xdribble_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL:
        for case_number_xdribble_skill_probs_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL:
            TP_counter, FP_counter, FN_counter, TN_counter = 0, 0, 0, 0
            for _, xdribble_row_data in xdribble_data_test.iterrows():
                actual_result_id = xdribble_row_data["result_id"]
                original_event_id = xdribble_row_data["original_event_id"]
                related_player_id = xdribble_row_data["player_id"]
                xdribble_skill_train_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == related_player_id][feature_names_xdribble_skill_probs_model]
                if (not xdribble_skill_train_data.empty):
                    xdribble_difficulty_value = MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID[(case_number_xdribble_model, original_event_id)]
                    xdribble_skill_value = MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID[(case_number_xdribble_skill_probs_model, related_player_id)]
                    predicted_result_id = 1 if (xdribble_difficulty_value * xdribble_skill_value) >= DEFINED_THRESHOLD else 0
                    if (actual_result_id == 1) and (predicted_result_id == 1):
                        TP_counter += 1
                    elif (actual_result_id == 0) and (predicted_result_id == 1):
                        FP_counter += 1
                    elif (actual_result_id == 1) and (predicted_result_id == 0):
                        FN_counter += 1
                    elif (actual_result_id == 0) and (predicted_result_id == 0):
                        TN_counter += 1

            print(f'True Positive : {TP_counter} ; False Positive : {FP_counter} ; False Negative : {FN_counter} ; True Negative : {TN_counter}')
            accuracy_with_player_skill = (TP_counter + TN_counter) / (TP_counter + TN_counter + FP_counter + FN_counter)
            accuracy_without_player_skill = MAPS_ACCURACY_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model]
            precision_with_player_skill = (TP_counter / (TP_counter + FP_counter)) if (TP_counter + FP_counter) > 0 else 0
            precision_without_player_skill = MAPS_PRECISION_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model]
            recall_with_player_skill = (TP_counter / (TP_counter + FN_counter)) if (TP_counter + FN_counter) > 0 else 0
            recall_without_player_skill = MAPS_RECALL_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model]
            f_score_with_player_skill = ((2 * precision_with_player_skill * recall_with_player_skill) / (precision_with_player_skill + recall_with_player_skill)) if (precision_with_player_skill + recall_with_player_skill) > 0 else 0
            f_score_without_player_skill = MAPS_F_SCORE_XDRIBBLE_MODEL[case_number_xdribble_model]

            maps_new_row = {
                "case_number_xdribble_model" : case_number_xdribble_model,
                "case_number_xdribble_skill_probs_model" : case_number_xdribble_skill_probs_model,
                "accuracy_without_skill" : accuracy_without_player_skill,
                "accuracy_with_player_skill" : accuracy_with_player_skill,
                "accuracy_difference" : (accuracy_without_player_skill - accuracy_with_player_skill),
                "precision_without_skill" : precision_without_player_skill,
                "precision_with_player_skill" : precision_with_player_skill,
                "precision_difference" : (precision_without_player_skill - precision_with_player_skill),
                "recall_without_skill" : recall_without_player_skill,
                "recall_with_player_skill" : recall_with_player_skill,
                "recall_difference" : (recall_without_player_skill - recall_with_player_skill),
                "f_score_without_skill" : f_score_without_player_skill,
                "f_score_with_player_skill" : f_score_with_player_skill,
                "f_score_difference" : (f_score_without_player_skill - f_score_with_player_skill)
            }
            new_row = pd.DataFrame(maps_new_row, index=[0])
            empty_test_result = pd.concat([new_row, empty_test_result.loc[:]]).reset_index(drop=True)
            
            maps_new_row_confusion_matrix = {
                "case_number_xdribble_model" : case_number_xdribble_model,
                "case_number_xdribble_skill_probs_model" : case_number_xdribble_skill_probs_model,
                "true_positive" : TP_counter,
                "true_negative" : TN_counter,
                "false_positive" : FP_counter,
                "false_negative" : FN_counter
            }
            new_row_confusion_matrix = pd.DataFrame(maps_new_row_confusion_matrix, index=[0])
            empty_confusion_matrix_result = pd.concat([new_row_confusion_matrix, empty_confusion_matrix_result.loc[:]]).reset_index(drop=True)

    filename_experiment_result = f'xdribble_final_accuracy_with_threshold_{DEFINED_THRESHOLD}.csv'
    empty_test_result.to_csv(DIRECTORY_MODEL + filename_experiment_result)
    filename_experiment_confusion_matrix_result = f'xdribble_confusion_matrix_with_threshold_{DEFINED_THRESHOLD}.csv'
    empty_confusion_matrix_result.to_csv(DIRECTORY_MODEL + filename_experiment_confusion_matrix_result)

True Positive : 26543 ; False Positive : 434 ; False Negative : 0 ; True Negative : 0


In [47]:
# # PLOT PRECISION-RECALL CURVE
# CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL = [1, 2, 3, 4, 5, 6]
# CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL = [1, 2]
# MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID = {}
# MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID = {}
# INCLUDE_SKILL = True
# COLUMNS_OPTIMUM_THRESHOLD_AND_AUC_SCORE_RESULT = [
#     "case_number_xdribble_model",
#     "case_number_xdribble_skill_probs_model",
#     "optimum_threshold",
#     "auc_score"
# ]
# DIRECTORY_MODEL = "data/model_xdribble/"
# FILENAME_DATA_TEST = "xdribble_dataset_for_accuracy_test.csv"

# # Difficulty Value : Change all numeric columns with StandardScaler
# scaler = preprocessing.StandardScaler()
# columns_minmax_scaler = ["distance_opponent", "num_opponent_in_path",
#                         "distance_receiver_opponent", "num_opponent_in_path_receiver",
#                         "num_opponent_0_and_45_before_midpoint_actor",
#                         "num_opponent_0_and_45_after_midpoint_actor",
#                         "num_opponent_0_and_45_before_midpoint_receiver",
#                         "num_opponent_0_and_45_after_midpoint_receiver"]
# big_dataframe_xdribble_model[columns_minmax_scaler] = scaler.fit_transform(big_dataframe_xdribble_model[columns_minmax_scaler])
# big_dataframe_xdribble_model = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'].isin([0,1])]

# # Player Skill Probabilities : Change all numeric columns with MinMaxScaler
# scaler = preprocessing.StandardScaler()
# columns_minmax_scaler = player_skills_column_included + player_attribute_column_included + ["statistic_success_action_probs"]
# xdribble_player_skill_probs_df[columns_minmax_scaler] = scaler.fit_transform(xdribble_player_skill_probs_df[columns_minmax_scaler])

# xdribble_data_test = big_dataframe_xdribble_model

# # Construct maps xdribble difficulty values and maps accuracy score xdribble model
# for case_number_xdribble_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL:
#     # Loads xdribble difficulty model
#     filename_xdribble_model = f'xdribble_model_case_{case_number_xdribble_model}.sav'
#     xdribble_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_model, 'rb'))
#     feature_names_xdribble_model = xdribble_model.feature_names_in_

#     actual_result_ids = pd.Series(xdribble_data_test["result_id"]).tolist()
#     original_event_ids = pd.Series(xdribble_data_test["original_event_id"]).tolist()
#     player_ids = pd.Series(xdribble_data_test["player_id"]).tolist()
#     xdribble_train_data = xdribble_data_test[feature_names_xdribble_model]
#     xdribble_difficulty_value = [p[1] for p in xdribble_model.predict_proba(xdribble_train_data)]
    
#     for idx, event_id in enumerate(original_event_ids):
#         actual_result_id = actual_result_ids[idx]
#         xdribble_difficulty_this_value = xdribble_difficulty_value[idx]
#         player_id = player_ids[idx]
#         player_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == player_id]
#         if (not player_data.empty):
#             MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID[(case_number_xdribble_model, event_id)] = xdribble_difficulty_this_value

# # Construct maps xdribble player skill probabilities value
# unique_player_ids = list(set(pd.Series(xdribble_data_test["player_id"]).tolist()))
# for case_number_xdribble_skill_probs_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL:
#     # Load xdribble player skill probs model
#     filename_xdribble_skill_probs_model = f'xdribble_player_skill_probs_model_case_{case_number_xdribble_skill_probs_model}.sav'
#     xdribble_skill_probs_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_skill_probs_model, 'rb'))
#     # feature_names_xdribble_skill_probs_model = xdribble_skill_probs_model.get_booster().feature_names
#     feature_names_xdribble_skill_probs_model = xdribble_skill_probs_model.feature_names_in_

#     for player_id in unique_player_ids:
#         xdribble_skill_train_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == player_id][feature_names_xdribble_skill_probs_model]
#         if (not xdribble_skill_train_data.empty):
#             xdribble_skill_value = xdribble_skill_probs_model.predict(xdribble_skill_train_data)[0]
#             xdribble_skill_value = 1 / (1 + math.exp(-1 * xdribble_skill_value))
#             MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID[(case_number_xdribble_skill_probs_model, player_id)] = xdribble_skill_value

# # Final Calculation and Saving Test Result
# empty_threshold_auc_result = pd.DataFrame(columns=COLUMNS_OPTIMUM_THRESHOLD_AND_AUC_SCORE_RESULT, index=[0])
# empty_threshold_auc_result_with_skill = pd.DataFrame(columns=COLUMNS_OPTIMUM_THRESHOLD_AND_AUC_SCORE_RESULT, index=[0])
# list_color_12_models = [
#     "#b53a1b", "#ef041a", "#f8b25d",
#     "#51c1e3", "#3d21a9", "#b8f56a",
#     "#ed5154", "#7d5f40", "#561294",
#     "#32075f", "#7000aa", "#65c5c6"
# ]
# for case_number_xdribble_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_MODEL:
#     for case_number_xdribble_skill_probs_model in CASE_NUMBER_LIST_FOR_AVAILABLE_XDRIBBLE_SKILL_PROBS_MODEL:
#         list_actual_result_values = []
#         list_predicted_result_values_without_skill = []
#         list_predicted_result_values_include_skill = []
#         for _, xdribble_row_data in xdribble_data_test.iterrows():
#             actual_result_id = xdribble_row_data["result_id"]
#             original_event_id = xdribble_row_data["original_event_id"]
#             related_player_id = xdribble_row_data["player_id"]
#             xdribble_skill_train_data = xdribble_player_skill_probs_df[xdribble_player_skill_probs_df["player_id"] == related_player_id][feature_names_xdribble_skill_probs_model]
#             if (not xdribble_skill_train_data.empty):
#                 xdribble_difficulty_value = MAPS_XDRIBBLE_DIFFICULTY_VALUES_BY_ORIGINAL_EVENT_ID[(case_number_xdribble_model, original_event_id)]
#                 xdribble_skill_value = MAPS_XDRIBBLE_SKILL_PROBS_VALUES_BY_PLAYER_ID[(case_number_xdribble_skill_probs_model, related_player_id)]
#                 list_actual_result_values.append(actual_result_id)
#                 list_predicted_result_values_without_skill.append(xdribble_difficulty_value)
#                 list_predicted_result_values_include_skill.append(xdribble_difficulty_value * xdribble_skill_value)
        
#         with_skill_precisions, with_skill_recalls, with_skill_thresholds = metrics.precision_recall_curve(list_actual_result_values, list_predicted_result_values_include_skill)
#         without_skill_precisions, without_skill_recalls, without_skill_thresholds = metrics.precision_recall_curve(list_actual_result_values, list_predicted_result_values_without_skill)
        
#         # UNCOMMENT THIS SNIPPET IF NEEDED !!
#         fscores = (2 * without_skill_precisions * without_skill_recalls) / (without_skill_precisions + without_skill_recalls)
#         idx_max = argmax(fscores)
#         auc_pr_curve = metrics.auc(without_skill_recalls, without_skill_precisions)
#         maps_new_row = {
#             "case_number_xdribble_model" : case_number_xdribble_model,
#             "case_number_xdribble_skill_probs_model" : case_number_xdribble_skill_probs_model,
#             "optimum_threshold" : without_skill_thresholds[idx_max],
#             "auc_score" : auc_pr_curve 
#         }
#         new_row = pd.DataFrame(maps_new_row, index=[0])
#         empty_threshold_auc_result = pd.concat([new_row, empty_threshold_auc_result.loc[:]]).reset_index(drop=True)

#         fscores = (2 * with_skill_precisions * with_skill_recalls) / (with_skill_precisions + with_skill_recalls)
#         idx_max = argmax(fscores)
#         auc_pr_curve = metrics.auc(with_skill_recalls, with_skill_precisions)
#         maps_new_row = {
#             "case_number_xdribble_model" : case_number_xdribble_model,
#             "case_number_xdribble_skill_probs_model" : case_number_xdribble_skill_probs_model,
#             "optimum_threshold" : with_skill_thresholds[idx_max],
#             "auc_score" : auc_pr_curve 
#         }
#         new_row = pd.DataFrame(maps_new_row, index=[0])
#         empty_threshold_auc_result_with_skill = pd.concat([new_row, empty_threshold_auc_result_with_skill.loc[:]]).reset_index(drop=True)

#         # UNCOMMENT THIS SNIPPET IF NEEDED !!
#         # label = f'Model number difficulty : {case_number_xdribble_model}. Model number skill : {case_number_xdribble_skill_probs_model}'
#         # final_color = list_color_12_models[0]
#         # del list_color_12_models[0]
#         # # plt.axis([0.9, 1, 0.8, 0.9])
#         # plt.plot(without_skill_recalls, without_skill_precisions, marker=".", color=final_color, label=label)
#         # plt.scatter(without_skill_recalls[idx_max], without_skill_precisions[idx_max], marker='o', color='black', label='Best')

#         # label = f'Model number difficulty : {case_number_xdribble_model}. Model number skill : {case_number_xdribble_skill_probs_model}'
#         # final_color = list_color_12_models[0]
#         # del list_color_12_models[0]
#         # # plt.axis([0.9, 1, 0.8, 0.9])
#         # plt.plot(with_skill_recalls, with_skill_precisions, marker=".", color=final_color, label=label)
#         # plt.scatter(with_skill_recalls[idx_max], with_skill_precisions[idx_max], marker='o', color='black', label='Best')

# # UNCOMMENT THIS SNIPPET IF NEEDED !!
# filename_threshold_auc_result = f'xdribble_threshold_auc_without_skill.csv'
# empty_threshold_auc_result.to_csv(DIRECTORY_MODEL + filename_threshold_auc_result)
# filename_threshold_auc_result = f'xdribble_threshold_auc_include_skill.csv'
# empty_threshold_auc_result_with_skill.to_csv(DIRECTORY_MODEL + filename_threshold_auc_result)

# # UNCOMMENT THIS SNIPPET IF NEEDED !!
# # plt.ylabel("Precision")
# # plt.xlabel("Recall")
# # plt.title("Precision-Recall Curve (XDribble Model)")
# # plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
# # plt.show()


In [48]:
# MAPS_CASE_NUMBER_MODEL = {
#     "best" : {
#         "case_number_xdribble_model" : 2,
#         "case_number_xdribble_skill_probs_model" : 2
#     }
# }
# MAPS_COMPETITIONS_AND_EVENT_ID_DIRECTORY = {
#     (55, 43) : {
#         "competition_name" : "UEFA Euro 2020"
#     },
#     (43, 106) : {
#         "competition_name" : "FIFA World Cup 2022"
#     }
# }
# DIRECTORY_MODEL = "data/model_xdribble/"

# # Helper Functions
# def calculate_standardscaler_value(row, column_name, std_value, mean_value):
#     return (row[column_name] - mean_value) / std_value

# def convert_all_numeric_columns_with_standard_scaler(df_data, column_names, df_description_numeric):
#     for column_name in column_names:
#         std_value_this_column = df_description_numeric[column_name]["std"]
#         mean_value_this_column = df_description_numeric[column_name]["mean"]
#         df_data[column_name] = df_data.apply(lambda x : calculate_standardscaler_value(x, column_name, std_value_this_column, mean_value_this_column), axis=1)
#     return df_data

# def calculate_minmaxscaler_value(row, column_name, min_value, max_value):
#     return ((row[column_name] - min_value) / (max_value - min_value))

# def convert_all_numeric_columns_with_min_max_scaler(df_data, column_names, df_description_numeric):
#     for column_name in column_names:
#         min_value_this_column = df_description_numeric[column_name]["min"]
#         max_value_this_column = df_description_numeric[column_name]["max"]
#         df_data[column_name] = df_data.apply(lambda x : calculate_minmaxscaler_value(x, column_name, min_value_this_column, max_value_this_column), axis=1)
#     return df_data

# def preprocess_player_skill_probs_df(player_skill_probs_df):
#     # 1. Change all numeric columns with MinMaxScaler
#     columns_minmax_scaler = player_attribute_column_included + player_skills_column_included
#     # Load Description for all Numeric Columns from External CSV
#     filename = 'xdribble_skill_probs_description_numeric_data.csv'
#     df_description_numeric = pd.read_csv(DIRECTORY_MODEL + filename, index_col=0)
#     # Preprocess to minmax scaler
#     player_skill_probs_df = convert_all_numeric_columns_with_standard_scaler(player_skill_probs_df, columns_minmax_scaler, df_description_numeric)

#     return player_skill_probs_df

# # Preprocessed for xdribble difficulty dataset
# scaler = preprocessing.StandardScaler()
# columns_minmax_scaler = ["distance_opponent", "num_opponent_in_path",
#                         "distance_receiver_opponent", "num_opponent_in_path_receiver",
#                         "num_opponent_0_and_45_before_midpoint_actor",
#                         "num_opponent_0_and_45_after_midpoint_actor",
#                         "num_opponent_0_and_45_before_midpoint_receiver",
#                         "num_opponent_0_and_45_after_midpoint_receiver"]
# big_dataframe_xdribble_model[columns_minmax_scaler] = scaler.fit_transform(big_dataframe_xdribble_model[columns_minmax_scaler])
# big_dataframe_xdribble_model = big_dataframe_xdribble_model[big_dataframe_xdribble_model['result_id'].isin([0,1])]

# # Calculate ratings for each players for shooting
# for competition_id, season_id in list(MAPS_COMPETITIONS_AND_EVENT_ID_DIRECTORY.keys()):
#     competition_name = MAPS_COMPETITIONS_AND_EVENT_ID_DIRECTORY[(competition_id, season_id)]["competition_name"]
#     filename_maps_competition_and_event_ids = f'xdribble_map_competition_and_event_id_for_{competition_name}.csv'
#     maps_competition_and_event_ids_df = pd.read_csv(DIRECTORY_MODEL + filename_maps_competition_and_event_ids)
#     list_related_event_ids_this_competition = pd.Series(maps_competition_and_event_ids_df['event_id_related']).tolist()
    
#     filename_vaep_ranking_this_competition = f'vaep_players_ranking_for_competition_{competition_name}.csv'
#     directory_vaep_ranking = "data/players_skill_dataset/"
#     vaep_ranking_df_this_competition = pd.read_csv(directory_vaep_ranking + filename_vaep_ranking_this_competition)

#     for scenario in list(MAPS_CASE_NUMBER_MODEL.keys()):
#         case_number_xdribble_model = MAPS_CASE_NUMBER_MODEL[scenario]["case_number_xdribble_model"]
#         case_number_xdribble_skill_model = MAPS_CASE_NUMBER_MODEL[scenario]["case_number_xdribble_skill_probs_model"]
#         empty_df_result = pd.DataFrame(columns=player_skills_column_included + player_attribute_column_included + ['statistic_dribble_contribution'], index=[0])

#         for _, row_player in vaep_ranking_df_this_competition.iterrows():
#             player_id = row_player['player_id']
#             maps_event_id_with_difficulty_value = {}
#             related_events_with_this_player = big_dataframe_xdribble_model \
#                     [(big_dataframe_xdribble_model['original_event_id'].isin(list_related_event_ids_this_competition)) & 
#                     (big_dataframe_xdribble_model['player_id'] == player_id)]
#             event_ids_list_for_this_player = pd.Series(related_events_with_this_player['original_event_id']).tolist()
            
#             # Give score for difficulty
#             filename_xdribble_model = f'xdribble_model_case_{case_number_xdribble_model}.sav'
#             xdribble_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_model, 'rb'))
#             feature_names_xdribble_model = xdribble_model.feature_names_in_
#             if (not related_events_with_this_player.empty):
#                 train_data = related_events_with_this_player[feature_names_xdribble_model]
#                 xdribble_difficulty_value = [p[1] for p in xdribble_model.predict_proba(train_data)]
#                 for idx, event_id in enumerate(event_ids_list_for_this_player):
#                     maps_event_id_with_difficulty_value[event_id] = 1 - xdribble_difficulty_value[idx]
            
#             # Give score for player skill
#             filename_xdribble_skill_probs_model = f'xdribble_player_skill_probs_model_case_{case_number_xdribble_skill_model}.sav'
#             xdribble_skill_probs_model = pickle.load(open(DIRECTORY_MODEL + filename_xdribble_skill_probs_model, 'rb'))
#             # feature_names_xdribble_skill_probs_model = xdribble_skill_probs_model.get_booster().feature_names
#             feature_names_xdribble_skill_probs_model = xdribble_skill_probs_model.feature_names_in_
#             train_data = pd.DataFrame(columns=player_skills_column_included + player_attribute_column_included, index=[0])
#             row_train_data = {}
#             for skill in player_skills_column_included:
#                 row_train_data[skill] = row_player[skill]
#             for attribute in player_attribute_column_included:
#                 row_train_data[attribute] = row_player[attribute]
#             row_train_data = pd.DataFrame(row_train_data, index=[0])
#             train_data = pd.concat([row_train_data, train_data.loc[:]]).reset_index(drop=True)
#             train_data.dropna(inplace=True)
#             train_data = preprocess_player_skill_probs_df(train_data)
#             train_data = train_data[feature_names_xdribble_skill_probs_model]
#             xdribble_skill_value = xdribble_skill_probs_model.predict(train_data)[0]
#             xdribble_skill_value = 1 / (1 + math.exp(-1 * xdribble_skill_value))

#             # Calculate final player contribution for this competition
#             total_score_this_player = 0
#             if (not related_events_with_this_player.empty):
#                 for _, row_event in related_events_with_this_player.iterrows():
#                     total_score_this_player += row_event['result_id'] * maps_event_id_with_difficulty_value[row_event['original_event_id']] * xdribble_skill_value
#             row_statistic_result = {}
#             row_statistic_result['player_id'] = row_player['player_id']
#             row_statistic_result['full_name'] = row_player['full_name']
#             for skill in player_skills_column_included:
#                 row_statistic_result[skill] = row_player[skill]
#             for attribute in player_attribute_column_included:
#                 row_statistic_result[attribute] = row_player[attribute]
#             row_statistic_result['statistic_dribble_contribution'] = total_score_this_player
#             row_statistic_result = pd.DataFrame(row_statistic_result, index=[0])
#             empty_df_result = pd.concat([row_statistic_result, empty_df_result.loc[:]]).reset_index(drop=True)

#         filename_player_score_result = f'xdribble_contribution_score_player_{scenario}_scenario_{competition_name}.csv'
#         empty_df_result.to_csv(DIRECTORY_MODEL + filename_player_score_result)
        